# Corona Chatbot

### Author: Harsh Khatri

In this project, I tried to create a closed domain chatbot. While extracting the data for the chatbot from the internet, I noticed that the questions are short, but the answers are lengthy.
If I try to build a seq-to-seq model for this chatbot, it might not work correctly bcoz of the length difference in questions and answers. The chatbot won't be able to predict the long responses accurately.
So I decided to treat every answer as a class. I trained my model to perform an LSTM multi-class classification.  The model is trained on different questions and trying to predict which answer class it belongs to.

PS: The bot is currently in development

In [2]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder

In [3]:
dataset = pd.read_excel("COVID-19-dataset.xlsx")
lastRow = 41
df = dataset.iloc[0:lastRow-1,:]

In [4]:
df.head(10)

,que,ans,alternate-que,reference
0,What is a coronavirus,Coronaviruses are a large family of viruses th...,What is a coronavirus%Tell me about coronaviru...,NaN
1,Can humans become infected with a novel corona...,Several known coronaviruses are circulating in...,Can humans become infected with a novel corona...,NaN
2,Are health workers at risk from a novel corona...,"Yes, they can be, as health care workers come ...",Are health workers at risk from a novel corona...,NaN
3,What is COVID-19,COVID-19 is the infectious disease caused by t...,What is COVID-19%Tell me about COVID19,NaN
4,Is COVID-19 the same as SARS,"No, SARS was more deadly but much less infecti...",Is COVID-19 the same as SARS%Is novel coronavi...,NaN
5,What are the symptoms of COVID-19,The most common symptoms of COVID-19 are: feve...,What are the symptoms of COVID-19%COVID19 symp...,NaN
6,Can I catch COVID-19 from surfaces or packages,Yes it is possible to catch COVID-19 from surf...,Can I catch COVID-19 from surfaces or packages...,NaN
7,Is it safe to receive a package from any area...,Yes. The likelihood of an infected person cont...,Is it safe to receive a package from any area...,NaN
8,What can I do to protect myself from COVID-19,You can reduce your chances of being infected ...,What can I do to protect myself from COVID-19,NaN
9,What should I do if I have visited an area whe...,If you have recently visited (past 14 days) ar...,What should I do if I have visited an area whe...,NaN


In [5]:
def make_ans_dictionary(ans_array):
    ans_dictionary = {}
    for i, item in enumerate(ans_array):
        ans_dictionary[i+1] = item
    return ans_dictionary
    
ans_dictionary = make_ans_dictionary(df["ans"].values)

In [6]:
def clean_text(text):
    text = text.lower()
    re.sub(r"covid-19", "covid19", text)
    re.sub(r"covid 19", "covid19", text)
    re.sub(r"novel coronavirus", "covid19", text)
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [7]:
def process_input_string(text):
    text = clean_text(text)
    text_arr = []
    for word in text.split():
        if word in word2int:
            text_arr.append(word2int[word])
        else:
            text_arr.append(0)
    
    padded_text = pad_sequences([np.array(text_arr)], maxlen=18, padding='post')
    final_text = np.reshape(padded_text, (padded_text.shape[0], padded_text.shape[1], 1))
    return final_text

In [8]:
def make_que_dictionary(que_array):
    que_dictionary = {}
    for i, item in enumerate(que_array):
        que_arr = []
        for text in item.split('%'):
            que_arr.append(clean_text(text))
        que_dictionary[i+1] = que_arr
    
    return que_dictionary

que_dictionary = make_que_dictionary(df["alternate-que"].values)

In [9]:
def training_dataset(que_dictionary):
    que_train = []
    y_train = []
    for i, item in que_dictionary.items():
        for que in item:
            que_train.append(que)
            y_train.append(i)
    
    return np.array(que_train).reshape(-1,1), np.array(y_train)
    
que_train, y_train = training_dataset(que_dictionary)
print(que_train[:6])

[['what is a coronavirus']
 ['tell me about coronavirus']
 ['what are coronaviruses']
 ['what is coronavirus']
 ['can humans become infected with a novel coronavirus of animal source']
 ['can i get affected by my dog']]


In [10]:
def modify_ytrain(y_train):
    encoder = OneHotEncoder()
    return encoder.fit_transform(y_train.reshape(-1,1)).toarray()

y_train = modify_ytrain(y_train)
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
def word_count(train):
    word2count = {}
    for text in train:
        for word in text.split():
            if word in word2count:
                word2count[word] += 1
            else:
                word2count[word] = 1
    return word2count

word2count = word_count(que_train[:,0])

In [12]:
def word_to_id(word2count, threshold):
    word2int = {}
    counter = 1
    for word, count in word2count.items():
        if count > threshold:
            word2int[word] = counter
            counter += 1
    return word2int
    
word2int = word_to_id(word2count, 0)

In [13]:
def id_to_word(word2int):
    int2word = {}
    for word, i in word2int.items():
        int2word[i] = word
    return int2word

int2word = id_to_word(word2int)

In [14]:
def que_to_int(que_train, word2int):
    que_array = []
    for que in que_train[:,0]:
        ints = []
        for word in que.split():
            ints.append(word2int[word])
        que_array.append(ints)
    return np.array(que_array)

que_array = que_to_int(que_train, word2int)

In [15]:
def pad_input_array(que_array, max_length):
    padded = pad_sequences(que_array, maxlen=max_length, padding='post')
    return np.array(padded)

X_train = pad_input_array(que_array, 18)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [16]:
def create_model(X_train, y_train, nepochs, batch_size):
    
    classifier = Sequential()
    classifier.add(LSTM(units=20, return_sequences=True, input_shape=(X_train.shape[1],1)))
    classifier.add(Dropout(0.2))
    classifier.add(LSTM(units=20, return_sequences=False))
    classifier.add(Dense(units=40, activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy')
    print(classifier.summary())
    classifier.fit(X_train, y_train, epochs = nepochs, batch_size=batch_size)
    
    return classifier

In [17]:
# Initializing parameters
no_of_epochs = 150
batch_size = 4

In [18]:
model = create_model(X_train, y_train, no_of_epochs, batch_size)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 18, 20)            1760      
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 20)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                840       
Total params: 5,880
Trainable params: 5,880
Non-trainable params: 0
_________________________________________________________________
None

Epoch 1/150
54/54 [==============================] - 1s 18ms/step - loss: 3.6910
Epoch 2/150
54/54 [==============================] - 0s 4ms/step - loss: 3.6595
Epoch 3/150
54/54 [==============================] - 0s 4ms/step - loss: 3.6459
Epo

54/54 [==============================] - 0s 4ms/step - loss: 1.3172
Epoch 89/150
54/54 [==============================] - 0s 3ms/step - loss: 1.3319A: 0s - loss: 1.365
Epoch 90/150
54/54 [==============================] - 0s 4ms/step - loss: 1.3021
Epoch 91/150
54/54 [==============================] - 0s 4ms/step - loss: 1.3850
Epoch 92/150
54/54 [==============================] - 0s 4ms/step - loss: 1.3481
Epoch 93/150
54/54 [==============================] - 0s 4ms/step - loss: 1.2977
Epoch 94/150
54/54 [==============================] - 0s 3ms/step - loss: 1.3424
Epoch 95/150
54/54 [==============================] - 0s 4ms/step - loss: 1.2895
Epoch 96/150
54/54 [==============================] - 0s 3ms/step - loss: 1.2987
Epoch 97/150
54/54 [==============================] - 0s 4ms/step - loss: 1.3006
Epoch 98/150
54/54 [==============================] - 0s 4ms/step - loss: 1.3083
Epoch 99/150
54/54 [==============================] - 0s 4ms/step - loss: 1.3091
Epoch 100/150
54/54 [=

In [33]:
def run_bot():    
    print('\033[1m' +"Enter 'Bye' to exit \n")
    print('\033[0m')
    while(True):
        question = input("You: ")
        if "bye" in question.lower():
            break
        que_array = process_input_string(question)
        pred_array = model.predict(que_array)[0]
        pred = np.argmax(pred_array)
        # print(np.max(pred_array))
        if np.max(pred_array) > 0.6:
            ans = ans_dictionary[pred+1]
        else:
            ans = "Sorry the bot coundn't answer that. Ask a different question."
        print('ChatBot: ' + ans +'\n')

In [34]:
run_bot()

Enter 'Bye' to exit 


You: what is covid-19
ChatBot: COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019

You: Symptoms of covid-19
ChatBot: The most common symptoms of COVID-19 are: fever, tiredness and dry cough

You: Should i wear a mask
ChatBot: Sorry the bot coundn't answer that. Ask a different question.

You: what should I not do
ChatBot: Sorry the bot coundn't answer that. Ask a different question.

You: bye
